In [2]:
import string
from threading import local
from matplotlib import pyplot as plt
import numpy as np
import cv2
import json
import numpy as np
import os
from statistics import median, mode,mean
import matplotlib.image as img
from matplotlib.ticker import PercentFormatter
from matplotlib import colors
import pandas as pd
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde
import scipy.stats as sts


In [3]:
right_image_Width=1000
right_image_height=420

In [4]:
def return_distance_ref(rawfile_path,raw_file_list,data_required ):
    distance_ref=[]
    for x in range(len(data_required)):

        with open(rawfile_path+raw_file_list[x], 'rb') as f:
            disparity_image = f.read()
        local_distance=[]
        d = [[0 for y in range(int(data_required[x]['TgtWidth']))]for a in range(int(data_required[x]['TgtHeight']))]
        for j in range(int(data_required[x]['TgtYPos_LeftUp']),int(data_required[x]['TgtYPos_LeftUp'])+int(data_required[x]['TgtHeight'])):
            for i in range(int(data_required[x]['TgtXPos_LeftUp']),int(data_required[x]['TgtXPos_LeftUp'])+int(data_required[x]['TgtWidth'])):
                disparity_j = int((right_image_height - j - 1) / 4)  # y-coordinate
                disparity_i = int(i / 4)  # x-coordinate
                # Load the disparity map
                # print((disparity_j * 256 + disparity_i) * 2)
                disparity =  disparity_image[(disparity_j * 256 + disparity_i) * 2] # integer

                disparity += disparity_image[(disparity_j * 256 + disparity_i) * 2 + 1] / 256 # decimal

                if disparity > 0: 
                    distance =  560 / (disparity - data_required[x]['inf_DP'])
                    local_distance.append(distance)
        #             d[j-int(data_required[x]['TgtYPos_LeftUp'])][i-int(data_required[x]['TgtXPos_LeftUp'])]=distance

        # heatmap = plt.pcolor(d)
        # plt.colorbar(heatmap)
        # plt.show()





        # # fig, axs = plt.subplots(1,1, figsize=(10,7), tight_layout=True)
        # h,e = np.histogram(local_distance, bins=100, density=True)
        # z = np.linspace(e.min(), e.max())

        # plot the histogram
        # plt.figure(figsize=(8,6))
        # plt.bar(e[:-1], h, width=np.diff(e), ec='k', align='edge', label='histogram')

        # plot the real KDE
        kde = sts.gaussian_kde(local_distance)
        # plt.plot(x, kde.pdf(x), c='C1', lw=8, label='KDE')

        # resample the histogram and find the KDE.
        # resamples = np.random.choice((e[:-1] + e[1:])/2, size=n*5, p=h/h.sum())
        # rkde = sts.gaussian_kde(resamples)

        # plot the KDE
        height = kde.pdf(local_distance)
        mode_value = local_distance[np.argmax(height)]
        distance_ref.append(mode_value)
    return distance_ref

right_image_Width=1000
right_image_height=420

In [5]:
from tqdm import tqdm
annotation_path = "../../complete_test_annotations_2/"
dir_list = os.listdir(annotation_path)
dir_list.sort()
for inddir in tqdm(dir_list):
    if(int(inddir[0:3])>93):
        f = open(annotation_path+inddir)
        data = json.load(f)
        df_annotation_list=pd.json_normalize(data)
        disparity_path="../test_videos/"+inddir[0:3]+"/disparity/"
        raw_file_list = os.listdir(disparity_path)
        raw_file_list.sort()
        # print(len(data))
        # print(len(raw_file_list))
        local_distance = return_distance_ref(disparity_path,raw_file_list,data)
        df_annotation_list['Distance_ref']=local_distance
        result = df_annotation_list.to_json('../../complete_test_annotations_2/'+inddir,orient='records')
    #

100%|██████████| 239/239 [8:23:18<00:00, 126.35s/it]    


In [6]:
df_annotation_list

,OwnSpeed,StrDeg,inf_DP,TgtXPos_LeftUp,TgtYPos_LeftUp,TgtWidth,TgtHeight,Distance_ref
0,42.8,0.4,5.769287,417.0,35.0,139.0,182.0,19.576509
1,42.9,0.7,5.769287,417.0,37.0,139.0,180.0,19.576509
2,43.1,0.7,5.769287,413.0,37.0,146.0,181.0,19.749107
3,43.2,1.0,5.769287,414.0,34.0,144.0,180.0,19.836552
4,43.3,1.0,5.769287,411.0,32.0,148.0,180.0,19.924775
...,...,...,...,...,...,...,...,...
138,50.5,-0.6,5.769287,392.0,22.0,176.0,182.0,15.783223
139,50.3,-0.3,5.769287,395.0,25.0,171.0,176.0,15.672791
140,50.2,0.2,5.769287,395.0,22.0,171.0,178.0,15.563894
141,50.0,0.2,5.769287,391.0,27.0,174.0,173.0,15.456500
